In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses

import matplotlib.pyplot as plt

import numpy as np

In [ ]:
class CGAN:
    def __init__(self):
        img_rows = 28
        img_cols = 28
        img_channels = 1
        self.img_shape = (img_rows, img_cols, img_channels)
        self.latent_dim = 100
        self.num_classes = 10
        
        optimizer = Adam(lr = 0.0002, beta_1 = 0.5, beta_2 = 0.999)
        
        # build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminnator.compile(loss = losses.binary_crossentropy, optimizer = optimizer, metrics = ['accuracy'])
        
        # build the generator
        self.generator = self.build_generator
        
        z = Input(shape = (self.latent_dim, ))
        target_label = Input(shape = (1, ))
        
        img_ = self.generator([z, label])
        
        # For the combined model we will only train the generator
        self.discriminator.trainable = False
        
        validity = self.discriminator(img_, label)
        
        self.combined = Model(inputs = [z, target_label], outputs = validity)
        self.combined.compile(loss = losses.binary_crossentropy, optimizer = optimizer)
        
    def build_discriminator(self):
        model = Sequential()

        model.add(Dense(512, input_dim=np.prod(self.img_shape)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        label = Input(shape=(1,), dtype='int32')

        label_embedding = Flatten()(Embedding(self.num_classes, np.prod(self.img_shape))(label))
        flat_img = Flatten()(img)

        model_input = multiply([flat_img, label_embedding])

        validity = model(model_input)

        return Model([img, label], validity)
    
    def build_generator